In [1]:
from gutenberg.acquire import load_etext
from gutenberg.cleanup import strip_headers
import pandas as pd 

In [2]:
df_metadata = pd.read_csv('gutenberg_metadata.csv')


Seleccionamos los autores que están en el top 100
https://www.gutenberg.org/browse/scores/top

In [8]:
authors = pd.read_csv('authors.csv')
#find first '(' and remove everything after
authors['Author'] = authors['Author'].str.split('(').str[0]

#split authors by ',' and inverse order and trim
authors['Author'] = authors['Author'].str.split(',').str[::-1]
authors['Author'] = authors['Author'].str.join(' ')
#trim
authors['Author'] = authors['Author'].str.strip()

authors

,Author
0,William Shakespeare
1,T. Smollett
2,E. M. Forster
3,L. M. Montgomery
4,Louisa May Alcott
...,...
95,Francisco Balagtas
96,Emperor of Rome Marcus Aurelius
97,P. G. Wodehouse
98,Edith Wharton


In [9]:
#select subset of books from authors
df_metadata = df_metadata[df_metadata['Author'].isin(authors['Author'])]
df_metadata


,Title,Author,Link,Bookshelf
